In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta
from tradingview_ta import *
print(tradingview_ta.__version__)

3.3.0


In [3]:
sasa = TA_Handler(
    symbol="SASA",
    screener="turkey",
    exchange="BIST",
    interval=Interval.INTERVAL_1_DAY
)
print(sasa.get_analysis().summary)

{'RECOMMENDATION': 'BUY', 'BUY': 14, 'SELL': 5, 'NEUTRAL': 7}


In [4]:
analysis = sasa.get_analysis()
ind = analysis.indicators

In [5]:
analysis = get_multiple_analysis(
        screener="turkey", 
        interval=Interval.INTERVAL_1_WEEK, 
        symbols=["bist:sasa", "bist:orma", "bist:sumas", "bist:aksa", "bist:petkm"]
        )

In [6]:
# EMA5 >= SMA10, MACD.macd > MACD.signal, Ichimoku.BLine < close, ADX+DI >= ADX-DI, SMA10 < close, Stoch.K > Stoch.D 
def indicator_strategy_1(stock_ind: dict, symbol: str):
    pattern1 = stock_ind['EMA5'] >= stock_ind['SMA10']
    pattern2 = stock_ind['MACD.macd'] > stock_ind['MACD.signal']
    pattern3 = stock_ind['Ichimoku.BLine'] < stock_ind['close']
    pattern4 = stock_ind['ADX+DI'] >= stock_ind['ADX-DI']
    pattern5 = stock_ind['SMA10'] < stock_ind['close']
    pattern6 = stock_ind['Stoch.K'] > stock_ind['Stoch.D']
    if(pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6):
        print(f'Buy the stock {symbol}')

In [7]:
for k in analysis:
    dict_ind = analysis[k].indicators
    indicator_strategy_1(dict_ind, k)

Buy the stock BIST:SASA
Buy the stock BIST:ORMA
Buy the stock BIST:SUMAS
Buy the stock BIST:AKSA


In [19]:
import datetime 
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

api_id=os.getenv('TG_api_id')
api_hash = os.getenv('TG_hash_id')
phone = os.getenv('TG_phone')
username = os.getenv('TG_username')

In [ ]:
from telethon.sync import TelegramClient

client = TelegramClient(None, api_id, api_hash)

async def main():
    chats = ['...']
    chat = '...'
    df = pd.DataFrame()
    async for message in client.iter_messages(chat, limit=1000):
        data = {"group": chat, "sender": message.sender_id, "text": message.text, "date": message.date}
        temp_df = pd.DataFrame(data)
        df = df.append(temp_df)

    df['date'] = df['date'].dt.tz_localize(None)
    print(df.head())
    df.to_csv('./burda.csv')

async with client:
    client.loop.run_until_complete(main())

In [18]:
import nest_asyncio
nest_asyncio.apply()